In [1]:
# Je ne sais pas vous, mais personnellement j'ai un écran large et ça m'agace
# d'avoir le notebook juste au milieu
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# librairies mathématiques et de gestion
import pandas as pd
import pickle

# On charge les dataframes :
 - df, qui contient toutes les données incident et mobilisation
 - station_coord, qui contient les coordonnées des stations présentes dans les données

In [3]:
# j'ai passé les data en csv, avec un nom un peu moins pénible
# la lecture doit se faire avec latin-1 pour certain caractère spéciaux
df2 = pd.read_csv('data/full_incident.csv',sep=',', encoding='latin-1')
df2.info()

C:\Users\Roman\anaconda3\envs\ptorch\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309284 entries, 0 to 1309283
Data columns (total 8 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   IncidentNumber    1309284 non-null  object
 1   IncidentGroup     1309283 non-null  object
 2   DateOfCall        1309284 non-null  object
 3   CalYear           1309284 non-null  int64 
 4   TimeOfCall        1309284 non-null  object
 5   HourOfCall        1309284 non-null  int64 
 6   Easting_rounded   1309284 non-null  int64 
 7   Northing_rounded  1309284 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 79.9+ MB


In [4]:
df = pd.read_csv('data/full_mobilisation.csv',sep=',', encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2032481 entries, 0 to 2032480
Data columns (total 5 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   IncidentNumber            object
 1   TimeMobilised             object
 2   TimeMobile                object
 3   TimeArrived               object
 4   DeployedFromStation_Name  object
dtypes: object(5)
memory usage: 77.5+ MB


# On peut maintenant réunir les données sur la base de l'identifiant de l'incident

In [5]:
df = df.merge(df2, on='IncidentNumber')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994634 entries, 0 to 1994633
Data columns (total 12 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   IncidentNumber            object
 1   TimeMobilised             object
 2   TimeMobile                object
 3   TimeArrived               object
 4   DeployedFromStation_Name  object
 5   IncidentGroup             object
 6   DateOfCall                object
 7   CalYear                   int64 
 8   TimeOfCall                object
 9   HourOfCall                int64 
 10  Easting_rounded           int64 
 11  Northing_rounded          int64 
dtypes: int64(4), object(8)
memory usage: 197.8+ MB


# Au final, on nettoies les valeurs manquantes dans les colonnes essentielles à la problématique (id, heure de l'appel, heure d'arrivée, coordonnées)

In [6]:
nb_null = df.isnull().sum()
print(nb_null)

IncidentNumber                  0
TimeMobilised                   0
TimeMobile                  45478
TimeArrived                 98204
DeployedFromStation_Name      665
IncidentGroup                   1
DateOfCall                      0
CalYear                         0
TimeOfCall                      0
HourOfCall                      0
Easting_rounded                 0
Northing_rounded                0
dtype: int64


In [7]:
df = df.dropna(subset=df.columns[[2, 3, 4]])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1864458 entries, 1 to 1994633
Data columns (total 12 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   IncidentNumber            object
 1   TimeMobilised             object
 2   TimeMobile                object
 3   TimeArrived               object
 4   DeployedFromStation_Name  object
 5   IncidentGroup             object
 6   DateOfCall                object
 7   CalYear                   int64 
 8   TimeOfCall                object
 9   HourOfCall                int64 
 10  Easting_rounded           int64 
 11  Northing_rounded          int64 
dtypes: int64(4), object(8)
memory usage: 184.9+ MB


# On peut enfin sauvegarder le dataset en csv

In [8]:
df.to_csv(r'data/Full_dataset.csv', index = False)